In [1]:
import numpy as np
from scipy.optimize import minimize

In [2]:
np.random.seed(42)

# MLE of Normal distribution
* Let $X \sim N(\mu, \sigma^2)$. 
    * Estimator vector $\bold{\theta} = (\mu, \sigma)$, Random samples $X_1, X_2, ...,X_n$

## 1. Likelihood Function
* Likelihood function = Joint distribution of random samples $X_1, X_2, ... , X_n$
$$ L(\mu, \sigma) = f(x_1, x_2, ...,x_n) = \Pi^{n}_{i=1}f(x_i;\theta)  $$

* Log Likelihood function
$$ logL(\mu, \sigma) = l(\mu, \sigma)= -\frac{n}{2}log2\pi-nlog\sigma-\frac{1}{2}\sum^n_{i=1}(\frac{x_i-\mu}{\sigma})^2 $$

In [3]:
# define the negative log-likelihood function for a normal distribution
def neg_log_likelihood(params, data):
    mu, sigma = params
    n = len(data)
    return (n/2) * np.log(2*np.pi*sigma**2) + (1/(2*sigma**2)) * np.sum((data - mu)**2)

## 2. Random Sampling
* Generate a sample dataset from normal distribution (``np.random.normal(mu, sigma, size)``)
* Define initial parameter values (``initial_params``) for optimization. These values serve as a starting point for the optimization algorithm.

In [4]:
# generate a sample dataset
data = np.random.normal(5, 2, 100) # X ~ N(5, 4)

# define the initial parameter values for optimization
initial_params = [0, 1]

## 3. Find MLE 
* Using the ``minimize()`` function from the ``scipy.optimize`` module, we minimize the negative log-likelihood function to find the maximum likelihood estimates (MLE) for the parameters.
* ``scipy.optimize.minimize(fun, x0, args=(), method=None, ...)``
    * fun: The objective function to be minimized. It should take the variables to be optimized as input and return the value of the function to be minimized.
    * x0: The initial guess or starting point for the optimization. It can be a scalar or an array-like object.
    * args: Additional arguments to be passed to the objective function fun.
    * method: The optimization algorithm to be used. It can be specified as a string or an OptimizeResult object. If not specified, the default algorithm is used.

In [5]:
# minimize the negative log-likelihood function to find MLE
result = minimize(neg_log_likelihood, initial_params, args=(data,))
mu_mle, sigma_mle = result.x

In [6]:
# print the estimated MLE
print("Estimated MLE - mu:", mu_mle) # True mu: 5
print("Estimated MLE - sigma:", sigma_mle) # True sigma: 2

Estimated MLE - mu: 4.792306853229411
Estimated MLE - sigma: 1.8072323306279252


# MLE for Linear Regression Model 
* Let $Y_i = \alpha + \beta(x_i -\bar{x}) + e_i$ where $\bar{x} = \frac{1}{n} \sum^{n}_{i=1}x_i$ and $e_i \sim^{iid} N(0, \sigma^2)$
    * Estimator vector $\bold{\theta} = (\alpha, \beta, \sigma)$, Random samples $X_1, X_2, ...,X_n$

## 1. Generate data using Linear regression model

In [7]:
# define linear regression model function
def reg(params, data):
    alpha, beta, sigma = params
    n = len(data)
    
    x_bar = np.mean(data)
    epsilon = np.random.normal(0, sigma, n)
    
    return alpha + beta*(data-x_bar) + epsilon 

In [8]:
# generate a sample dataset (x, y) using linear regression
x = np.random.randn(100) 
true_params = [2, 3, 1]
y = reg(true_params, x) # yi

## 2. Likelihood Function
* Negative Log Likelihood function
$$ -logL(\alpha, \beta, \sigma) = \frac{n}{2} log(2\pi\sigma^2) + \frac{\sum^n_{i=1}[y_i-\alpha-\beta(x_i-\bar{x})]^2}{2\sigma^2} $$

* cf. Line-point vertical distance function (Residual)
$$H(\alpha, \beta) = \sum^n_{i=1}[y_i-\alpha-\beta(x_i-\bar{x})]^2 = \sum^n_{i=1} (y_i-\hat{y_i})^2$$


In [9]:
# define the negative log-likelihood function for linear regression
def neg_log_likelihood(params, x, y):
    alpha, beta, sigma = params 
    x_bar = np.mean(x)
    n = len(x)
    
    y_pred = alpha + beta * (x - x_bar) # yhat; no epsilon term 
    residuals = y - y_pred
    
    return (n/2 * np.log(2*np.pi*sigma**2)) + (np.sum(residuals**2) / 2*sigma**2)

## 3. Find MLE 

In [10]:
# set the initial parameter values for optimization
initial_params = [0, 0, 1]  # initial values for alpha, beta, and sigma

In [11]:
# minimize the negative log-likelihood function
result = minimize(neg_log_likelihood, initial_params, args=(x, y))
alpha_mle, beta_mle, sigma_mle = result.x

In [12]:
# print the estimated MLEs
print("Estimated MLE - alpha:", alpha_mle) # True alpha: 2
print("Estimated MLE - beta:", beta_mle) # True beta: 3
print("Estimated MLE - sigma:", sigma_mle) # True sigma: 1

Estimated MLE - alpha: 1.8740702012081738
Estimated MLE - beta: 2.632050426560142
Estimated MLE - sigma: -0.949647150130543


# MLE for Bivariate Normal Distribution

* Starting from the most basic model $\boldsymbol{Z} = \{Z_1, Z_2\}'$, where $Z_1, Z_2$ independent, and follow $N(0, 1)$
    * $\boldsymbol{Z} \sim N(\boldsymbol{\mu}, \boldsymbol{\Sigma}) = N(\{\mu_1,\mu_2\}', \begin{pmatrix} Var(Z_1) & Cov(Z_1, Z_2) \\ Cov(Z_2, Z_1) & Var(Z_2) \end{pmatrix}) = N(\{0,0\}', \begin{pmatrix} 1 & 0 \\ 0 & 1 \end{pmatrix})$ 

## 1. Generate data from Bivariate normal distribution

In [189]:
n = 10000 # number of data points
N = 2

In [190]:
mu_true = np.array([0, 0])
sigma_true = np.array([[1, 0], [0, 1]])

data = np.random.multivariate_normal(mu_true, sigma_true, size=n) 

In [191]:
def check_matrix_properties(matrix):
    # check if the matrix is symmetric
    is_symmetric = np.allclose(matrix, matrix.T)
    
    # check if the matrix is positive semi-definite
    eigenvalues = np.linalg.eigvals(matrix)
    is_positive_semidef = np.all(eigenvalues >= 0)
    
    # check if the matrix is invertible
    is_invertible = np.linalg.matrix_rank(matrix) == matrix.shape[0]
    
    return is_symmetric, is_positive_semidef, is_invertible

In [192]:
is_symmetric, is_positive_semidef, is_invertible = check_matrix_properties(sigma_true)
print("Is symmetric:", is_symmetric)
print("Is positive semi-definite:", is_positive_semidef)
print("Is invertible:", is_invertible)

Is symmetric: True
Is positive semi-definite: True
Is invertible: True


In [193]:
data[0]

array([-1.54066973,  0.8305306 ])

## 2. Likelihood function

In [194]:
np.array([[initial_params[N], initial_params[-1]], [initial_params[-1], initial_params[N+1]]])

array([[1, 0],
       [0, 1]])

In [195]:
# define the log-likelihood function for the multivariate normal distribution
def log_likelihood(params, data):
    mu = np.array(params[:N]) # [1.0, 2.0]
    Sigma = np.array([[params[N]**1, params[-1]], [params[-1], params[N+1]**1]])
    n = len(data)
    
    inv_Sigma = np.linalg.inv(Sigma)
    log_det_Sigma = np.log(np.linalg.det(Sigma))
    
    logL = 0.0
    for i in range(n):
        x = data[i]
        logL += -0.5 * (x - mu) @ inv_Sigma @ (x - mu).T - 0.5 * log_det_Sigma
    return -logL

In [196]:
# # initialize the mean vector and covariance matrix
# mu_initial = np.array([1.0, 2.0])
# sigma_initial = np.array([[1.0, 0.0], [0.0, 1.0]])

true_params = [0,0,1,1,0]
initial_params = true_params
# initial_params = [1.0, 2.0, 1.0, 1.0, 0.0]

# combine the mean vector and covariance matrix
# initial_params = np.concatenate((mu_initial, sigma_initial.flatten()))

In [197]:
is_symmetric, is_positive_semidef, is_invertible = check_matrix_properties(sigma_params)
print("Is symmetric:", is_symmetric)
print("Is positive semi-definite:", is_positive_semidef)
print("Is invertible:", is_invertible)

Is symmetric: True
Is positive semi-definite: True
Is invertible: True


In [198]:
# minimize the negative log-likelihood to estimate the parameters
result = minimize(log_likelihood, initial_params, args=(data,))
estimated_params = result.x

In [199]:
estimated_params

array([ 0.00734159, -0.01058431,  0.99620288,  1.00486874, -0.01024977])

In [200]:
# extract the estimated mean and covariance matrix
estimated_mean = estimated_params[:N]
estimated_cov = estimated_params[N:].reshape((N, N))

print("Estimated mean:", estimated_mean)
print("Estimated covariance matrix:")
print(estimated_cov)

ValueError: cannot reshape array of size 3 into shape (2,2)

# MLE for Multivariate Normal Distribution

* Let $\boldsymbol{X} \sim N(\boldsymbol{\mu}, \boldsymbol{\Sigma})$
* N: size of dimension, n: number of data points
    * $\boldsymbol{\mu}$: n 1 $\times$ N vector = (1, N) array
    * $\boldsymbol{\sigma}$: n N $\times$ N matrix = (N, N) array
    * Random vector $\bold{X} = \{X_1, X_2, ...,X_n\}'$ where $X_i \sim^{iid} N(\mu, \Sigma)$:  n 1 $\times$ N vector = (n, 1, N) array


## 1. Generate data from Multivariate normal distribution

In [ ]:
N = 2 # size of dimension
n = 100  # number of data points

In [ ]:
def generate_covariance_matrix(N):
    # generate random matrix of size N by N
    random_matrix = np.random.rand(N, N)
    # create a symmetric covariance matrix
    covariance_matrix = np.dot(random_matrix, random_matrix.T)
    return covariance_matrix

In [ ]:
mu_true = np.array([0., 1.])
Sigma_true = generate_covariance_matrix(N)

data = np.random.multivariate_normal(mu_true, Sigma_true, size=n)

In [ ]:
data.shape # (n, N)

(100, 2)

## 2. Likelihood Function
* Log Likelihood function
$$ logL(\boldsymbol {\mu}, \boldsymbol {\Sigma})= l(\boldsymbol {\mu}, \boldsymbol {\Sigma}) = \sum^n_{i=1} -\frac{1}{2}(\bold{x}_i-\boldsymbol{\mu})^T\boldsymbol{\Sigma}^{-1}(\bold{x}_i-\boldsymbol{\mu})-{\frac {1}{2}}log|{\boldsymbol {\Sigma }}|$$

In [ ]:
# define the log-likelihood function for the multivariate normal distribution
def log_likelihood(params, data):
    mu = params[:N]
    Sigma = params[N:].reshape((N, N))
    n = len(data)
    
    inv_Sigma = np.linalg.inv(Sigma)
    log_det_Sigma = np.log(np.linalg.det(Sigma))
    
    logL = 0.0
    for i in range(n):
        x = data[i]
        logL += -0.5 * (x - mu).T @ inv_Sigma @ (x - mu) - 0.5 * log_det_Sigma
    return -logL

## 3. Find MLE

In [ ]:
# initialize the parameters
mean_params = np.zeros(N)
cov_params = np.eye(N).flatten()

initial_params = np.concatenate((mean_params, cov_params))

In [ ]:
# minimize the negative log-likelihood to estimate the parameters
result = minimize(log_likelihood, initial_params, args=(data,))
estimated_params = result.x

/var/folders/q0/pmjpggzn2yqbzlw4r8fbvcsm0000gn/T/ipykernel_85249/1427696438.py:8: RuntimeWarning: invalid value encountered in log
  log_det_Sigma = np.log(np.linalg.det(Sigma))
/var/folders/q0/pmjpggzn2yqbzlw4r8fbvcsm0000gn/T/ipykernel_85249/1427696438.py:8: RuntimeWarning: invalid value encountered in log
  log_det_Sigma = np.log(np.linalg.det(Sigma))
/var/folders/q0/pmjpggzn2yqbzlw4r8fbvcsm0000gn/T/ipykernel_85249/1427696438.py:8: RuntimeWarning: invalid value encountered in log
  log_det_Sigma = np.log(np.linalg.det(Sigma))


In [ ]:
# extract the estimated mean and covariance matrix
estimated_mean = estimated_params[:N]
estimated_cov = estimated_params[N:].reshape((N, N))

print("Estimated mean:", estimated_mean)
print("Estimated covariance matrix:")
print(estimated_cov)

Estimated mean: [-501.60599538 1047.45045075]
Estimated covariance matrix:
[[ -893.94250711  1619.92624854]
 [ 1619.92632273 -1762.87581142]]
